In [1]:
import psycopg2 as p
from datetime import datetime
import time, json

In [14]:
start_time = time.time()

In [86]:
password = '******'

conn = p.connect(database = 'postgres', user = 'postgres', password = password, 
                       host = 'localhost', port = 5433)

cur = conn.cursor()

# Step 1: deactivations based on old MCS150s or inactive DOTs in the MCMIS file

## OO Deactivation

In [22]:
# Based on missing census_num in MCMIS
# Automatically happens
cur.execute("""
update fleetseek.fleet
set 
    active_flag = 'N',
    update_date = now()::date

from (
    select m.census_num 
    from fleetseek.fleet as f
    left join mcmis as m
    on f.usdot = m.census_num
    where 
        m.census_num is null and
        f.active_flag = 'Y' and 
        f.product_code = 'OOD'
) as sub_q
where fleet.usdot = sub_q.census_num
""")
conn.commit()

In [23]:
# Outdated mcs150_date

cur.execute("""
update fleetseek.fleet
set 
    active_flag = 'N',
    update_date = now()::date

from (
    select m.census_num 
    from fleetseek.fleet as f
    join mcmis as m
    on f.usdot = m.census_num
    where 
        m.mcs150_date < now() - interval '30 months' and
        f.product_code = 'OOD' and
        f.active_flag = 'Y'
) as sub_q
where fleet.usdot = sub_q.census_num
""")

## FH Deactivation


In [24]:
# For FH fleets between 1-19 total_vehicles units based on missing census_num in MCMIS
cur.execute("""
update fleetseek.fleet
set 
    active_flag = 'N',
    update_date = now()::date
    
from ( 
    select m.census_num
    from fleetseek.fleet as f
    left join mcmis as m
    on f.usdot = m.census_num
    where 
        m.census_num is null and
        f.active_flag = 'Y' and 
        f.product_code = 'NMC' and
        f.total_vehicles between 1 and 19 and
        f.usdot != 0
) as sub_q
where fleet.usdot = sub_q.census_num
""")
conn.commit()

In [25]:
# FH fleets with old mcs150_date

cur.execute("""
update fleetseek.fleet
set 
    active_flag = 'N',
    update_date = now()::date

from ( 
    select m.census_num
    from fleetseek.fleet as f
    join mcmis as m
    on f.usdot = m.census_num
    where
        f.product_code = 'NMC' and
        f.total_vehicles between 1 and 19 and
        f.usdot != 0 and
        m.mcs150_date < now() - interval '30 months'
) as sub_q
where fleet.usdot = sub_q.census_num
""")

# Step 2: Reactivations

## OO Reinstate

In [26]:
#12/19/2019 first pass

# 1/23/2020 additions. Reinstate based on new mcs150_dates, will also capture DOTs. This will also automatically update fleet numbers for OO. 
# As of 1/27/2020 it takes less than 1 minute to update

cur.execute("""
update fleetseek.fleet
set 
    trucks_owned = sub_q.owntruck,
    tractors_owned = sub_q.owntract,
    trailers_owned = sub_q.owntrail,
    trucks_leased = sub_q.trucks_leased,
    tractors_leased = sub_q.tractors_leased,
    trailers_leased = sub_q.trailers_leased,
    total_trucks = sub_q.total_trucks,
    total_tractors = sub_q.total_tractors,
    total_trailers = sub_q.total_trailers,
    total_vehicles = sub_q.total_vehicles,
    total_vehicles_owned = sub_q.total_vehicles_owned,
    total_vehicles_leased = sub_q.total_vehicles_leased,
    dot_hmind = sub_q.hm_ind,
    year_mileage = sub_q.mcs150mileageyear,
    vehicle_miles = sub_q.mlg150,
    drivers = sub_q.tot_drs,
    cdl_drivers = sub_q.cdl_drivers,
    active_flag = 'Y'
    
from 
(
    select 
        census_num, 
        name, 
        owntruck, 
        owntract, 
        owntrail, 
        sum(trptruck+trmtruck) as trucks_leased, 
        sum(trptract+trmtract) as tractors_leased, 
        sum(trptrail+trmtrail) as trailers_leased, 
        sum(owntruck+trptruck+trmtruck) as total_trucks, 
        sum(owntract+trmtract+trptract) as total_tractors,
        sum(owntrail+trmtrail+trptrail) as total_trailers, 
        sum(owntruck+trptruck+trmtruck+owntract+trptract+trmtract) as total_vehicles,
        sum(owntruck+owntract) as total_vehicles_owned, 
        sum(trmtruck+trptruck+trmtract+trptract) as total_vehicles_leased, 
        hm_ind, 
        mcs150mileageyear, 
        mlg150, 
        tot_drs, 
        cdl_drs as cdl_drivers, 
        mcs150_date, 
        passengers, 
        class, 
        carship
    from mcmis
    group by 
        census_num, 
        name, 
        owntruck, 
        owntract, 
        owntrail, 
        hm_ind, 
        mcs150mileageyear, 
        mlg150, 
        tot_drs, 
        cdl_drs, 
        mcs150_date, 
        passengers, 
        class, 
        carship
) as sub_q

where
    fleet.usdot = sub_q.census_num and
    fleet.active_flag = 'N' and
    fleet.update_date < sub_q.mcs150_date and
    sub_q.mcs150_date > now() - interval '2 years' and
    fleet.product_code = 'OOD'
""")
conn.commit()

## OO Reinstate

In [27]:
# 1/30/2020 activating OO but not updating numbers if there's an increase greater than 50 

cur.execute("""
update fleetseek.fleet
set 
    active_flag = 'Y'
    
from 
(
    select 
        census_num, 
        name, 
        owntruck, 
        owntract, 
        owntrail, 
        sum(trptruck+trmtruck) as trucks_leased, 
        sum(trptract+trmtract) as tractors_leased, 
        sum(trptrail+trmtrail) as trailers_leased, 
        sum(owntruck+trptruck+trmtruck) as total_trucks, 
        sum(owntract+trmtract+trptract) as total_tractors,
        sum(owntrail+trmtrail+trptrail) as total_trailers, 
        sum(owntruck+trptruck+trmtruck+owntract+trptract+trmtract) as total_vehicles,
        sum(owntruck+owntract) as total_vehicles_owned, 
        sum(trmtruck+trptruck+trmtract+trptract) as total_vehicles_leased, 
        hm_ind, 
        mcs150mileageyear, 
        mlg150, 
        tot_drs, 
        cdl_drs as cdl_drivers, 
        mcs150_date, 
        passengers, 
        class, 
        carship
    from mcmis
    group by 
        census_num, 
        name, 
        owntruck, 
        owntract, 
        owntrail, 
        hm_ind, 
        mcs150mileageyear, 
        mlg150, 
        tot_drs, 
        cdl_drs, 
        mcs150_date, 
        passengers, 
        class, 
        carship
) as sub_q

where
    fleet.usdot = sub_q.census_num and
    fleet.active_flag = 'N' and
    fleet.update_date < sub_q.mcs150_date and
    sub_q.mcs150_date > now() - interval '2 years' and
    fleet.product_code = 'OOD' and
    sub_q.total_vehicles >= 51
""")
conn.commit()

## FH reinstate

In [30]:
# For-hire reinstatement based on original rules

cur.execute("""
update fleetseek.fleet
set 
    active_flag = 'Y'
from (
    select 
        census_num, 
        name, 
        owntruck, 
        owntract, 
        owntrail, 
        sum(trptruck+trmtruck) as trucks_leased, 
        sum(trptract+trmtract) as tractors_leased, 
        sum(trptrail+trmtrail) as trailers_leased, 
        sum(owntruck+trptruck+trmtruck) as total_trucks, 
        sum(owntract+trmtract+trptract) as total_tractors,
        sum(owntrail+trmtrail+trptrail) as total_trailers, 
        sum(owntruck+trptruck+trmtruck+owntract+trptract+trmtract) as total_vehicles,
        sum(owntruck+owntract) as total_vehicles_owned, 
        sum(trmtruck+trptruck+trmtract+trptract) as total_vehicles_leased, 
        hm_ind, 
        mcs150mileageyear, 
        mlg150, 
        tot_drs, 
        cdl_drs as cdl_drivers, 
        mcs150_date, 
        passengers, 
        class, 
        carship
    from mcmis
    group by census_num, name, owntruck, owntract, owntrail, hm_ind, mcs150mileageyear, mlg150, tot_drs, cdl_drs, mcs150_date, passengers, class, carship
) as sub_q
where 
    fleet.usdot = sub_q.census_num and
    fleet.active_flag = 'N' and
    fleet.update_date < sub_q.mcs150_date and
    sub_q.mcs150_date > now() - interval '2 years' and
    
        (sub_q.total_trucks >= 2 or 
        (sub_q.total_trucks = 1 and sub_q.total_tractors = 1) or
        (sub_q.total_trucks >= 1 and sub_q.total_tractors = 0 and sub_q.total_trailers >= 2) or
        (sub_q.total_trucks = 0 and sub_q.total_tractors >= 1 and sub_q.total_trailers >=2)) and
        
    sub_q.class in ('A', 'AB', 'ABD', 'ABG', 'ABL', 'AG', 'AGL', 'AL', 'B', 'BL') and
    fleet.product_code = 'NMC' and
    fleet.total_vehicles <= 50 and
    sub_q.total_vehicles <= 50

""")

## 2/20/2020 rewrite to only reinstate if less than 50 and doesn't increase by more than 50

In [30]:
# For-hire reinstatement based on original rules

cur.execute("""
update fleetseek.fleet
set 
    active_flag = 'Y'
from (
    select 
        m.census_num, 
        m.name, 
        m.owntruck, 
        m.owntract, 
        m.owntrail, 
        sum(m.trptruck+m.trmtruck) as mcmis_trucks_leased, 
        sum(m.trptract+m.trmtract) as mcmis_tractors_leased, 
        sum(m.trptrail+m.trmtrail) as mcmis_trailers_leased, 
        sum(m.owntruck+m.trptruck+m.trmtruck) as mcmis_total_trucks, 
        sum(m.owntract+m.trmtract+m.trptract) as mcmis_total_tractors,
        sum(owntrail+trmtrail+trptrail) as mcmis_total_trailers, 
        sum(owntruck+trptruck+trmtruck+owntract+trptract+trmtract) as mcmis_total_vehicles,
        sum(owntruck+owntract) as mcmis_total_vehicles_owned, 
        sum(trmtruck+trptruck+trmtract+trptract) as mcmis_total_vehicles_leased, 
        m.hm_ind, 
        m.mcs150mileageyear, 
        m.mlg150, 
        m.tot_drs, 
        m.cdl_drs as mcmis_cdl_drivers, 
        m.mcs150_date, 
        m.passengers, 
        m.class, 
        m.carship,
        f.total_vehicles,
        abs(f.total_vehicles - sum(m.owntruck + m.trmtruck + m.trptruck + m.owntract + m.trmtract + m.trptract)) as absolute_difference
    from mcmis as m
    join fleetseek.fleet as f
    on m.census_num = f.usdot
    group by census_num, name, owntruck, owntract, owntrail, hm_ind, mcs150mileageyear, mlg150, tot_drs, cdl_drs, mcs150_date, passengers, class, carship
) as sub_q
where 
    fleet.usdot = sub_q.census_num and
    fleet.active_flag = 'N' and
    fleet.update_date < sub_q.mcs150_date and
    sub_q.mcs150_date > now() - interval '2 years' and
    
        (sub_q.total_trucks >= 2 or 
        (sub_q.total_trucks = 1 and sub_q.total_tractors = 1) or
        (sub_q.total_trucks >= 1 and sub_q.total_tractors = 0 and sub_q.total_trailers >= 2) or
        (sub_q.total_trucks = 0 and sub_q.total_tractors >= 1 and sub_q.total_trailers >=2)) and
        
    sub_q.class in ('A', 'AB', 'ABD', 'ABG', 'ABL', 'AG', 'AGL', 'AL', 'B', 'BL') and
    fleet.product_code = 'NMC' and
    fleet.total_vehicles < 50 and
    absolute_difference < 50

""")

# Step 3a: All changes to fleets sizes. This will happen after initial activations/deactivations 



## Fleet table and fleet_primary

#### Fleets between 1 and 19

In [ ]:
# # The suppress_redundant_updates_trigger will prevent any repeat updates that don't actually change the rows
# # Updates both fleet and fleet_primary
# # This uses a common table expression (CTE) to update fleet_primary from updates that happen in the fleet table

# # 1/21/2020 update 

# # 1/29/2020 update: likely won't need this one but keeping just in case
# cur.execute("""
# with updated as (

#     update fleetseek.fleet
#     set 

#         trucks_owned = sub_q.owntruck,
#         tractors_owned = sub_q.owntract,
#         trailers_owned = sub_q.owntrail,
#         trucks_leased = sub_q.trucks_leased,
#         tractors_leased = sub_q.tractors_leased,
#         trailers_leased = sub_q.trailers_leased,
#         total_trucks = sub_q.total_trucks,
#         total_tractors = sub_q.total_tractors,
#         total_trailers = sub_q.total_trailers,
#         total_vehicles = sub_q.total_vehicles,
#         total_vehicles_owned = sub_q.total_vehicles_owned,
#         total_vehicles_leased = sub_q.total_vehicles_leased,
#         dot_hmind = sub_q.hm_ind,
#         year_mileage = sub_q.mcs150mileageyear,
#         vehicle_miles = sub_q.mlg150,
#         drivers = sub_q.tot_drs,
#         cdl_drivers = sub_q.cdl_drivers,
#         update_date = now()::date,
#         mcn
#     from 
#         (select 
#             census_num, 
#             name, 
#             owntruck, 
#             owntract, 
#             owntrail, 
#             sum(trptruck+trmtruck) as trucks_leased, 
#             sum(trptract+trmtract) as tractors_leased, 
#             sum(trptrail+trmtrail) as trailers_leased, 
#             sum(owntruck+trptruck+trmtruck) as total_trucks, 
#             sum(owntract+trmtract+trptract) as total_tractors,
#             sum(owntrail+trmtrail+trptrail) as total_trailers, 
#             sum(owntruck+trptruck+trmtruck+owntract+trptract+trmtract) as total_vehicles,
#             sum(owntruck+owntract) as total_vehicles_owned, 
#             sum(trmtruck+trptruck+trmtract+trptract) as total_vehicles_leased, 
#             hm_ind, 
#             mcs150mileageyear, 
#             mlg150, 
#             tot_drs, 
#             cdl_drs as cdl_drivers, 
#             mcs150_date, 
#             passengers, 
#             class, 
#             carship

#          from mcmis
#          group by owntruck, owntract, owntrail, mcs150_date, census_num, name, hm_ind, mcs150mileageyear, mlg150, tot_drs, cdl_drs, passengers, class, carship
#         ) as sub_q

#     where fleet.usdot = sub_q.census_num and
#     fleet.update_date < sub_q.mcs150_date and
#     sub_q.mcs150_date > now() - interval '2 years' and
#     fleet.active_flag = 'Y' and
#     sub_q.total_vehicles between 1 and 19 and
#     fleet.product_code in ('NMC', 'PFD')
#     returning
#         fleet.fleet_id,
#         fleet.total_vehicles,
#         fleet.total_trailers

# )

# update fleetseek.fleet_primary
# set
#     total_vehicles = updated.total_vehicles,
#     total_trailers = updated.total_trailers,
#     update_date = now()::date

# from updated
# where fleet_primary.fleet_id = updated.fleet_id

# """)
# conn.commit()

## Fleet table and fleet_primary

#### Fleets between 1 and 49 that don't increase by too much

In [31]:
# As of 1/29/2020 this will be the main one used

cur.execute("""

with updated as (
    
    update fleetseek.fleet
    set
        trucks_owned = sub_q.trucks_owned,
        trucks_leased = sub_q.trucks_leased,
        tractors_owned = sub_q.tractors_owned,
        tractors_leased = sub_q.tractors_leased,
        trailers_owned = sub_q.owntrail,
        trailers_leased = sub_q.trailers_leased,
        total_trucks = sub_q.total_trucks,
        total_tractors = sub_q.total_tractors,
        total_trailers = sub_q.total_trailers,
        total_vehicles = sub_q.total_vehicles,
        total_vehicles_owned = sub_q.total_vehicles_owned,
        total_vehicles_leased = sub_q.total_vehicles_leased,
        dot_hmind = sub_q.hm_ind,
        year_mileage = sub_q.mcs150mileageyear,
        vehicle_miles = sub_q.mlg150,
        drivers = sub_q.tot_drs,
        cdl_drivers = sub_q.cdl_drivers,
        update_date = now()::date,
        mcn = sub_q.mcn, 
        average_miles = sub_q.average_miles
        
        
    from 
        (select 
            m.census_num, 
            m.name, 
            sum(m.owntruck + f.haz_trucks_owned) as trucks_owned, 
            sum(m.trptruck + m.trmtruck + f.haz_trucks_leased) as trucks_leased, 
            m.owntract as tractors_owned, 
            sum(m.trptract + m.trmtract) as tractors_leased,
            sum(m.owntrail + f.haz_trailers_owned) as trailers_owned, 
            sum(m.trptrail + m.trmtrail + f.haz_trailers_leased) as trailers_leased, 
            sum(m.owntruck + m.trptruck + m.trmtruck + f.haz_trucks_owned + f.haz_trucks_leased) as total_trucks, 
            sum(m.owntract + m.trmtract + m.trptract) as total_tractors,
            sum(m.owntrail + m.trmtrail + m.trptrail + f.haz_trailers_owned + f.haz_trailers_leased) as total_trailers, 
            sum(m.owntruck + m.trptruck + m.trmtruck + m.owntract + m.trptract + m.trmtract + f.haz_trucks_owned + f.haz_trucks_leased) as total_vehicles,
            sum(m.owntruck + m.owntract + f.haz_trucks_owned) as total_vehicles_owned, 
            sum(m.trmtruck + m.trptruck + m.trmtract + m.trptract + f.haz_trucks_leased) as total_vehicles_leased, 
            m.hm_ind, 
            m.mcs150mileageyear, 
            m.mlg150, 
            m.tot_drs, 
            m.cdl_drs as cdl_drivers, 
            m.mcs150_date, 
            m.passengers, 
            m.class, 
            m.carship,
            m.icc1 as mcn,
            m.mlg150 / (nullif(sum(m.owntruck + m.trmtruck + m.trptruck + m.owntract + m.trmtract + m.trptract + sub_e.haz_trucks_owned + sub_e.haz_trucks_leased), 0)) as average_miles

         from mcmis as m
         join fleetseek.fleet as f
         on m.census_num = f.usdot
         join
         (-- subquery that returns fleets where there is a difference of 50 or less
            select
                m.census_num,
                f.fleet_id, 
                f.total_vehicles,
                f.product_code,
                f.active_flag,
                m.mcs150_date,
                f.update_date,
                f.haz_trucks_leased,
                f.haz_trucks_owned,

                sum(m.owntruck + m.trmtruck + m.trptruck + m.owntract + m.trmtract + m.trptract) as mcmis_total_vehicles,
                abs(f.total_vehicles - sum(m.owntruck + m.trmtruck + m.trptruck + m.owntract + m.trmtract + m.trptract)) as absolute_difference

            from fleetseek.fleet as f
            join mcmis as m
            on f.usdot = m.census_num
            where

                f.active_flag = 'Y' and
                m.mcs150_date > f.update_date and
                m.mcs150_date > now() - interval '2 years' and
                f.total_vehicles between 1 and 49 
            group by 
                f.fleet_id, 
                f.total_vehicles,
                f.product_code,
                f.active_flag,
                m.mcs150_date,
                f.update_date,
                m.owntruck,
                m.trmtruck,
                m.trptruck,
                m.owntract,
                m.trmtract,
                m.trptract,
                m.census_num
            having 
                abs(f.total_vehicles - sum(m.owntruck + m.trmtruck + m.trptruck + m.owntract + m.trmtract + m.trptract)) < 50
            ) as sub_e
            on m.census_num = sub_e.census_num
            group by 
                owntruck, 
                owntract, 
                owntrail, 
                m.mcs150_date, 
                m.census_num, 
                name, 
                hm_ind, 
                mcs150mileageyear, 
                mlg150, 
                tot_drs, 
                cdl_drs, 
                passengers, 
                class, 
                carship,
                icc1
        ) as sub_q
    where 
        fleet.usdot = sub_q.census_num and
        fleet.update_date < sub_q.mcs150_date and
        sub_q.mcs150_date > now() - interval '2 years' and
        fleet.active_flag = 'Y' and
        sub_q.total_vehicles between 1 and 49
        
    returning
        fleet.fleet_id,
        fleet.total_vehicles,
        fleet.total_trailers
)

update fleetseek.fleet_primary
set
    total_vehicles = updated.total_vehicles,
    total_trailers = updated.total_trailers,
    update_date = now()::date

from updated
where fleet_primary.fleet_id = updated.fleet_id

""")
conn.commit()

# Step 3b: deactivating fleets if they fall below thresholds

In [32]:
# Deactivating OO and NMC, happens before sending new addresses to Business Coder
cur.execute("""
update fleetseek.fleet
    set active_flag = 'N'
where 
    product_code in ('OOD', 'NMC') and
    total_tractors = 0 and
    total_trailers = 0 and
    total_trucks = 1 and
    active_flag = 'Y'
""")
conn.commit()

## Private Fleet deactivation (1/27/2020 still holding off on this)

In [ ]:
# #For PFD companies
# cur.execute("""
# update fleetseek.fleet
# set active_flag = 'N'
# from fleetseek.fleet as f
# where 
#     f.product_code = 'PFD' and
#     f.total_tractors = 1 and
#     f.total_trailers = 0 and
#     f.total_trucks <= 1 and 
#     f.total_vehicles <= 1 and
#     f.active_flag = 'Y'
# """)
# conn.commit()

# Step 3c: inserting to be sent to the Business Coder API

#### Will be for both auto and manual updates


In [87]:
# 1/21/2020 Initial write

# Won't be able to do Canadian or Mexican addresses since Business Coder only works for US addresses


cur.execute("""
insert into fleetseek.updates_bn_coder_output 

select f.fleet_id, f.usdot, m.name, m.phy_str, m.phy_city, m.phy_st, m.phy_zip, m.mai_str, m.mai_city, m.mai_st, m.mai_zip, m.tel_num, 

    case
        when (upper(left(replace(replace(replace(replace(l.address1, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) != upper(left(replace(replace(replace(m.phy_str, '-', ''), '&', ''), ' ', ''), 3)) and 
              upper(left(replace(replace(replace(replace(l2.address1, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) != upper(left(replace(replace(replace(m.mai_str, '-', ''), '&', ''), ' ', ''), 3))) then 'both'
              
        when (upper(left(replace(replace(replace(replace(l.address1, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) != upper(left(replace(replace(replace(m.phy_str, '-', ''), '&', ''), ' ', ''), 3)) and 
              upper(left(replace(replace(replace(replace(l2.address1, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) = upper(left(replace(replace(replace(m.mai_str, '-', ''), '&', ''), ' ', ''), 3))) then 'physical'
        
        when (upper(left(replace(replace(replace(replace(l.address1, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) = upper(left(replace(replace(replace(m.phy_str, '-', ''), '&', ''), ' ', ''), 3)) and 
              upper(left(replace(replace(replace(replace(l2.address1, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) != upper(left(replace(replace(replace(m.mai_str, '-', ''), '&', ''), ' ', ''), 3))) then 'mailing'
    end as differences,
    
    case
        when (m.phy_str = m.mai_str) then true else false
    end as uniform_mcmis_addresses
    
from mcmis as m
join fleetseek.fleet as f
on m.census_num = f.usdot 
join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
join fleetseek.location as l
on fp.primary_physical_location_id = l.location_id 
join fleetseek.location as l2
on fp.primary_mailing_location_id = l2.location_id
where
    (upper(left(replace(replace(replace(replace(l.address1, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) != upper(left(replace(replace(replace(m.phy_str, '-', ''), '&', ''), ' ', ''), 3)) or 
     upper(left(replace(replace(replace(replace(l2.address1, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) != upper(left(replace(replace(replace(m.mai_str, '-', ''), '&', ''), ' ', ''), 3))) and 
     m.mcs150_date > l.updated_date_time::date and
     m.mcs150_date > now() - interval '2 years' and
     f.active_flag = 'Y' and
     f.product_code != 'CMC' and
     f.product_code != 'MMC'

on conflict (fs_fleet_id) do nothing

""")
conn.commit()

### GVW Weight Class 8 update

In [37]:
cur.execute("""
update fleetseek.fleet
set 
    gvw_class_8 = 'Y'
where
    total_tractors > 0 and
    gvw_class_8 <> 'Y';
    

update fleetseek.fleet
set 
    gvw_class_8 = 'N'
where 
    total_tractors = 0 and
    gvw_class_8 <> 'N';

""")
conn.commit()

### Fleet trailer type 

In [53]:
# # Changes to the utility types but not for fleets with a hazmat indicator
# # Will use query below, but keeping just in case

# cur.execute("""
# update fleetseek.fleet_trailer_type
# set 
#     trailer_type_id = 142
# from ( 
#     select f.fleet_id
#     from fleetseek.fleet_trailer_type as ftt
#     join fleetseek.fleet as f
#     on ftt.fleet_id = f.fleet_id
#     join (
#         select 
#           m.census_num, m.owntract, m.trmtract, m.trptract, m.owntrail, m.trmtrail, m.trptrail, m.mcs150_date
#           from mcmis as m
#           group by m.census_num, m.owntract, m.trmtract, m.trptract, m.owntrail, m.trmtrail, m.trptrail, m.mcs150_date
#           having (sum(owntract+trmtract+trptract) = 0 and sum(owntrail+trmtrail+trptrail) > 0) 
#          ) as sub_a
#     on f.usdot = sub_a.census_num
#     where 
#         f.update_date::date = now()::date and 
#         sub_a.mcs150_date > now() - interval '2 years' and
#         f.dot_hmind != 'Y' and
#         f.active_flag = 'Y'
# ) as sub_q
# where fleet_trailer_type.fleet_id = sub_q.fleet_id

# """)
# conn.commit()

In [53]:
# Changes to the utility types but not for fleets with a hazmat indicator
# 2/26/2020 added in a CTE to update fleet_primary as well. This will be query that is kept

cur.execute("""
with updated as (
    update fleetseek.fleet_trailer_type
    set 
        trailer_type_id = 142
    from ( 
        select 
            f.fleet_id as fs_fleet_id
        from fleetseek.fleet_trailer_type as ftt
        join fleetseek.fleet as f
        on ftt.fleet_id = f.fleet_id
        where 
            f.dot_hmind != 'Y' and
            f.active_flag = 'Y' and
            f.total_tractors = 0 and
            f.total_trailers > 0
    ) as sub_q
    where 
        fleet_trailer_type.fleet_id = sub_q.fs_fleet_id and
        fleet_trailer_type.trailer_type_id <> 142
    returning 
        trailer_type_id, 
        fs_fleet_id
)

update fleetseek.fleet_primary
set 
    trailer_type_id = array[updated.trailer_type_id]
from updated
where 
    fleet_primary.fleet_id = updated.fs_fleet_id and
    array_to_string(fleet_primary.trailer_type_id, ',') <> '142'

""")
conn.commit()

# Step 4: changes in types after fleet size changes


### For-Hire to OO

In [54]:
#This query will happen after all the deactivations and fleet size changes

cur.execute("""
update fleetseek.fleet
set 
    product_code = 'OOD',
    fleet_type_id = 105,
    sic_code = 4218,
    company_type_code = 18
from (
    select 
        product_code as sub_q_product_code, 
        fleet_type_id, 
        sic_code, 
        total_vehicles, 
        total_tractors, 
        total_trailers, 
        total_trucks, 
        fleet_id
    from fleetseek.fleet
    where 
        total_tractors <= 1 and
        total_trailers <= 1 and
        total_vehicles <= 1 and
        total_trucks = 0 and
        product_code = 'NMC' and
        active_flag = 'Y'
    ) as sub_q
where fleet.fleet_id = sub_q.fleet_id and
product_code != 'OOD'
""")
conn.commit()

### OO to FH

In [61]:
# 1/27/2020 still need to figure out how to update sic_code
cur.execute("""
update fleetseek.fleet
set 
    product_code = 'NMC',
    fleet_type_id = 104
    
from ( 
    select *
    from fleetseek.fleet as f
    join mcmis as m
    on f.usdot = m.census_num
    where 
        (f.total_trucks >= 2 or 
        (f.total_trucks >= 1 and f.total_tractors >= 1) or
        (f.total_trucks >= 1 and f.total_tractors = 0 and f.total_trailers >= 2) or
        (f.total_trucks = 0 and f.total_tractors >= 1 and f.total_trailers >=2)) and
        m.class in ('A', 'AB', 'ABD', 'ABG', 'ABL', 'AG', 'AGL', 'AL', 'B', 'BL') and
        f.product_code = 'OOD' and
        f.active_flag = 'Y'
) as sub_q
where fleet.fleet_id = sub_q.fleet_id and
fleet.product_code != 'NMC'

""")

## Auto updating commodities 

In [ ]:
cur.execute("""
update fleetseek.fleet_primary
set
    commodity_id = sub_q.commodity_id

from (
    select string_to_array(sub_e.commodity_ids, ',')::int[] as commodity_id, census_num, fleet_id
     from (
         select string_agg(commodity_id::varchar, ', ') as commodity_ids, census_num, fleet_id
         from (
             select *
             from fleetseek.commodity as fc
             join 
             (
                select *
                from (   

                    select 
                        census_num, 
                        concat(genfreight, '  ', household, '  ', metalsheet, '  ', motorveh, '  ', drivetow, '  ', logpole, '  ', bldgmat, '  ', 
                                              mobilehome, '  ', machlrg, '  ', produce, '  ', liqgas, '  ', intermodal, '  ', passengers, '  ', oilfield, '  ', 
                                              livestock, '  ', grainfeed, '  ', coalcoke, '  ', meat, '  ', garbage, '  ', usmail, '  ', chem, '  ',
                                              drybulk, '  ', coldfood, '  ', beverages, '  ', paperprod, '  ', utility, '  ', farmsupp, '  ', construct, '  ',
                                              waterwell, '  ', othercargo) as all_commodities,
                        fleet_id
                    from (
                        select
                            f.fleet_id,
                            census_num,
                            case 
                                when(genfreight is not null) then 'General Freight' else null
                            end as genfreight,
                            case
                                when(household is not null) then 'Household Goods' else null
                            end as household,
                            case
                                when(metalsheet is not null) then 'Metal, Sheet, Coil, Rolls' else null
                            end as metalsheet,
                            case
                                when(motorveh is not null) then 'Motor Vehicles' else null
                            end as motorveh,
                            case
                                when(drivetow is not null) then 'Driveaway/Towaway' else null
                            end as drivetow,
                            case
                                when(logpole is not null) then 'Logs, Poles, Beams, Lumber' else null
                            end as logpole,
                            case
                                when(bldgmat is not null) then 'Building Materials' else null
                            end as bldgmat,
                            case
                                when(mobilehome is not null) then 'Mobile Homes' else null
                            end as mobilehome,
                            case
                                when(machlrg is not null) then 'Machinery, Large Objects' else null
                            end as machlrg,
                            case
                                when(produce is not null) then 'Fresh Produce' else null
                            end as produce,
                            case
                                when(liqgas is not null) then 'Liquids/Gases' else null
                            end as liqgas,
                            case
                                when(intermodal is not null) then 'Intermodal Containers' else null
                            end as intermodal,
                            case
                                when(passengers is not null) then 'Passengers' else null
                            end as passengers,
                            case
                                when(oilfield is not null) then 'Oilfield Equipment' else null
                            end as oilfield,
                            case
                                when(livestock is not null) then 'Livestock' else null
                            end as livestock,
                            case
                                when(grainfeed is not null) then 'Grain, Feed, Hay' else null
                            end as grainfeed,
                            case
                                when(coalcoke is not null) then 'Coal, Coke' else null
                            end as coalcoke,
                            case
                                when(meat is not null) then 'Meat' else null
                            end as meat,
                            case
                                when(garbage is not null) then 'Garbage, Refuse, Trash' else null
                            end as garbage,
                            case
                                when(usmail is not null) then 'U.S. Mail' else null
                            end as usmail,
                            case
                                when(chem is not null) then 'Chemicals' else null
                            end as chem,
                            case
                                when(drybulk is not null) then 'Commodities Dry Bulk' else null
                            end as drybulk,
                            case
                                when(coldfood is not null) then 'Refrigerated Food' else null
                            end as coldfood,
                            case
                                when(beverages is not null) then 'Beverages' else null
                            end as beverages,
                            case
                                when(paperprod is not null) then 'Paper Products' else null
                            end as paperprod,
                            case
                                when(utility is not null) then 'Utility' else null
                            end as utility,
                            case
                                when(farmsupp is not null) then 'Farm Supplies' else null
                            end as farmsupp,
                            case
                                when(construct is not null) then 'Construction' else null
                            end as construct,
                            case
                                when(waterwell is not null) then 'Water - Well' else null
                            end as waterwell,
                            initcap(othercargo) as othercargo
                        from mcmis as m
                        join fleetseek.fleet as f
                        on m.census_num = f.usdot
                        where
                            f.total_vehicles between 1 and 49 and
                            f.product_code in ('OOD', 'NMC') and
                            f.active_flag = 'Y' and
                            f.update_date = now()::date
                    ) as sub_a
                ) as sub_b, unnest(string_to_array(all_commodities, '  ')) as token
            ) as sub_c
            on fc.description = sub_c.token
            order by census_num, commodity_id
        ) as sub_d  
        group by sub_d.census_num, fleet_id
    ) as sub_e 
) as sub_q
where fleet_primary.fleet_id = sub_q.fleet_id

""")
conn.commit()

## Auto updating trailer types and SICs for OOs and FHs

In [ ]:
cur.execute("""

with updated as (
    update fleetseek.fleet_primary
    set
        trailer_type_id = sub_q.trailer_types
    from (
        select 
            string_to_array(sub_h.trailer_types, ', ')::int[] as trailer_types, 
            sub_h.census_num, 
            sub_h.sic,
            sub_h.fleet_id
        from (
            select 
                sub_g."SIC" as sic, 
                sub_g.census_num, 
                string_agg(sub_g.trailer_type_id::varchar, ', ') as trailer_types,
                sub_g.fleet_id
            from (
                select 
                    tt.*, 
                    sub_f.census_num, 
                    sub_f.trailer_types, 
                    sub_f."SIC",
                    sub_f.fleet_id
                from (
                    select 
                        string_to_array(sub_e.trailer_types, ' ') as trailer_types, 
                        sub_e.census_num, 
                        sub_e."SIC",
                        sub_e.fleet_id
                    from (
                        select 
                            string_agg(initcap(sub_d.trailer_type), ' ') as trailer_types, 
                            sub_d."SIC", 
                            sub_d.census_num,
                            sub_d.fleet_id
                        from (
                            select 
                                case 
                                    when (sub_c.total_tractors = 0 and total_trailers > 0) then 'Utility'
                                    when (sub_c.total_trailers = 1) then coalesce("Trailer Type if 1 Trailer")
                                    when (sub_c.total_trailers = 2) then concat(coalesce("Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"), ' ', coalesce("Trailer Type if 1 Trailer"))
                                    when (sub_c.total_trailers = 3) then concat(coalesce("Trailer Type if 3 Trailers", "Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"), ' ', coalesce("Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"), ' ', coalesce("Trailer Type if 1 Trailer"))
                                    when (sub_c.total_trailers = 4) then concat(coalesce("Trailer Type if 4 or more trailers", "Trailer Type if 3 Trailers", "Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"), ' ', coalesce("Trailer Type if 3 Trailers", "Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"), ' ', coalesce("Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"), ' ', coalesce("Trailer Type if 1 Trailer"))
                                    when (sub_c.total_trailers = 5) then concat(coalesce("Trailer Type if 5 or More trailers", "Trailer Type if 4 or more trailers", "Trailer Type if 3 Trailers", "Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"), ' ', coalesce("Trailer Type if 3 Trailers", "Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"), ' ', coalesce("Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"), ' ', coalesce("Trailer Type if 1 Trailer"))    
                                    when (sub_c.total_trailers = 6) then concat(coalesce("Trailer type if 6 or more trailers", "Trailer Type if 5 or More trailers", "Trailer Type if 4 or more trailers", "Trailer Type if 3 Trailers", "Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"), ' ', coalesce("Trailer Type if 3 Trailers", "Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"), ' ', coalesce("Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"), ' ', coalesce("Trailer Type if 1 Trailer"))
                                    when (sub_c.total_trailers = 7) then concat(coalesce("Trailer type if 7 or more trailers", "Trailer type if 6 or more trailers", "Trailer Type if 5 or More trailers", "Trailer Type if 4 or more trailers", "Trailer Type if 3 Trailers", "Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"), ' ', coalesce("Trailer Type if 3 Trailers", "Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"), ' ', coalesce("Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"), ' ', coalesce("Trailer Type if 1 Trailer"))
                                    when (sub_c.total_trailers > 7) then concat("Trailer type if 7 or more trailers", ' ', "Trailer type if 6 or more trailers", ' ', "Trailer Type if 5 or More trailers", ' ', "Trailer Type if 4 or more trailers", ' ', "Trailer Type if 3 Trailers", ' ', "Trailer Type if 2 Trailers", ' ', "Trailer Type if 1 Trailer")
                                end as trailer_type,
                                sub_c.*
                            from (
                                select 
                                    sub_b.total_trailers, 
                                    sub_b.fleet_id,
                                    sub_b.census_num, 
                                    sub_b.total_tractors, 
                                    sub_b.sub_b_commod_key, 
                                    tk.*
                                from (
                                    select 
                                        concat( genfreight,  household,  usmail,  intermodal,  paperprod,  beverages,  coldfood,  produce,  meat, 
                                             livestock,  grainfeed,  farmsupp,  drybulk,  bldgmat,  metalsheet,  logpole,  construct,  machlrg,  oilfield,
                                             waterwell,  liqgas,  chem,  coalcoke,  garbage,  drivetow,  motorveh,  mobilehome,  utility,  cargoothr) as sub_b_commod_key,
                                        *

                                    from (
                                        select
                                            census_num,
                                            fleet_id,
                                            abs(f.total_vehicles - sum(m.owntruck + m.trmtruck + m.trptruck + m.owntract + m.trmtract + m.trptract)) as absolute_difference,

                                            case 
                                                when(genfreight is not null) then 'genfreight' else null
                                            end as genfreight,
                                            case
                                                when(household is not null) then 'household' else null
                                            end as household,
                                            case
                                                when(metalsheet is not null) then 'metalsheet' else null
                                            end as metalsheet,
                                            case
                                                when(motorveh is not null) then 'motorveh' else null
                                            end as motorveh,
                                            case
                                                when(drivetow is not null) then 'drivetow' else null
                                            end as drivetow,
                                            case
                                                when(logpole is not null) then 'logpole' else null
                                            end as logpole,
                                            case
                                                when(bldgmat is not null) then 'bldgmat' else null
                                            end as bldgmat,
                                            case
                                                when(mobilehome is not null) then 'mobilehome' else null
                                            end as mobilehome,
                                            case
                                                when(machlrg is not null) then 'machlrg' else null
                                            end as machlrg,
                                            case
                                                when(produce is not null) then 'produce' else null
                                            end as produce,
                                            case
                                                when(liqgas is not null) then 'liqgas' else null
                                            end as liqgas,
                                            case
                                                when(intermodal is not null) then 'intermodal' else null
                                            end as intermodal,
                                            case
                                                when(passengers is not null) then 'passengers' else null
                                            end as passengers,
                                            case
                                                when(oilfield is not null) then 'oilfield' else null
                                            end as oilfield,
                                            case
                                                when(livestock is not null) then 'livestock' else null
                                            end as livestock,
                                            case
                                                when(grainfeed is not null) then 'grainfeed' else null
                                            end as grainfeed,
                                            case
                                                when(coalcoke is not null) then 'coalcoke' else null
                                            end as coalcoke,
                                            case
                                                when(meat is not null) then 'meat' else null
                                            end as meat,
                                            case
                                                when(garbage is not null) then 'garbage' else null
                                            end as garbage,
                                            case
                                                when(usmail is not null) then 'usmail' else null
                                            end as usmail,
                                            case
                                                when(chem is not null) then 'chem' else null
                                            end as chem,
                                            case
                                                when(drybulk is not null) then 'drybulk' else null
                                            end as drybulk,
                                            case
                                                when(coldfood is not null) then 'coldfood' else null
                                            end as coldfood,
                                            case
                                                when(beverages is not null) then 'beverages' else null
                                            end as beverages,
                                            case
                                                when(paperprod is not null) then 'paperprod' else null
                                            end as paperprod,
                                            case
                                                when(utility is not null) then 'utility' else null
                                            end as utility,
                                            case
                                                when(farmsupp is not null) then 'farmsupp' else null
                                            end as farmsupp,
                                            case
                                                when(construct is not null) then 'construct' else null
                                            end as construct,
                                            case
                                                when(waterwell is not null) then 'waterwell' else null
                                            end as waterwell,
                                            case
                                                when(cargoothr is not null) then 'cargoothr' else null
                                            end as cargoothr,
                                            sum(owntrail + trmtrail + trptrail) as total_trailers,
                                            sum(owntract + trmtract + trptract) as total_tractors
                                        from my_public.mcmis as m
                                        join fleetseek.fleet as f
                                        on m.census_num = f.usdot

                                        where 
                                            f.product_code in ('OOD', 'NMC') and
                                            f.active_flag = 'Y' and
                                            m.mcs150_date > f.update_date and
                                            m.mcs150_date > now() - interval '2 years' and
                                            f.total_vehicles between 1 and 49 

                                        group by 

                                            genfreight, household, metalsheet, motorveh, drivetow, logpole, bldgmat, 
                                            mobilehome, machlrg, produce, liqgas, intermodal, passengers, oilfield, 
                                            livestock, grainfeed, coalcoke, meat, garbage, usmail, chem,
                                            drybulk, coldfood, beverages, paperprod, utility, farmsupp, construct,
                                            waterwell, othercargo, census_num, cargoothr, f.total_vehicles, fleet_id
                                        having 
                                            abs(f.total_vehicles - sum(m.owntruck + m.trmtruck + m.trptruck + m.owntract + m.trmtract + m.trptract)) < 50 
                                    ) as sub_a
                                ) as sub_b
                                left join 
                                "Trailer_key" as tk
                                on btrim(tk.commod_key) = btrim(sub_b.sub_b_commod_key)
                            ) as sub_c
                        ) as sub_d
                        group by census_num, trailer_type, "SIC", fleet_id
                    ) as sub_e
                ) as sub_f
                join fleetseek.trailer_type as tt
                on tt.name = any(sub_f.trailer_types)	
                order by census_num, trailer_type_id
            ) as sub_g
            group by sub_g."SIC", sub_g.census_num, sub_g.fleet_id

        ) as sub_h
    ) as sub_q
    where fleet_primary.fleet_id = sub_q.fleet_id
    returning 
        sub_q.sic,
        sub_q.fleet_id
)
update fleetseek.fleet
set
    sic_code = updated.sic
from updated
where 
    fleet.fleet_id = updated.fleet_id

""")
conn.commit()

# Step 5: changes to phone numbers, contacts, location info (different script, will happen after sending to API), etc.

### 1/24/2020 initial write

### Phone_number changes



In [67]:
# Phone number changes for regular, toll-free, and fax
cur.execute("""

-- regular phone number
update fleetseek.phone_number
set
    phone_number = sub_q.tel_num,
    updated_date_time = now()
from (
    select 
        m.tel_num, 
        pn.phone_number_id
    from mcmis as m
    join fleetseek.fleet as f
    on m.census_num = f.usdot 
    join fleetseek.fleet_primary as fp
    on f.fleet_id = fp.fleet_id
    join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id
    where 
        m.tel_num != pn.phone_number and
        m.mcs150_date > pn.updated_date_time and
        m.mcs150_date > now() - interval '2 years' and
        f.active_flag = 'Y' and 
        f.total_vehicles between 1 and 19 and
        pn.phone_type_id = 1 and 
        m.tel_num not similar to ('800%|888%|877%|866%|855%|844%') and
        m.tel_num is not null
) as sub_q
where phone_number.phone_number_id = sub_q.phone_number_id;


-- toll free update
update fleetseek.phone_number
set
    phone_number = sub_q.tel_num,
    updated_date_time = now()
from (
    select 
        m.tel_num, 
        pn.phone_number_id
    from mcmis as m
    join fleetseek.fleet as f
    on m.census_num = f.usdot 
    join fleetseek.fleet_primary as fp
    on f.fleet_id = fp.fleet_id
    join fleetseek.phone_number as pn
    on fp.primary_toll_free_id = pn.phone_number_id
    where 
        m.tel_num != pn.phone_number and
        m.mcs150_date > f.update_date and
        m.mcs150_date > now() - interval '2 years' and
        f.active_flag = 'Y' and 
        f.total_vehicles between 1 and 19 and
        pn.phone_type_id = 2 and
        m.tel_num similar to ('800%|888%|877%|866%|855%|844%')
) as sub_q
where phone_number.phone_number_id = sub_q.phone_number_id;

-- fax number update

update fleetseek.phone_number
set
    phone_number = sub_q.fax_num,
    updated_date_time = now()
from (
    select 
        m.fax_num, 
        pn.phone_number,
        pn.phone_number_id,
        pn.phone_type_id
    from mcmis as m
    join fleetseek.fleet as f
    on m.census_num = f.usdot 
    join fleetseek.fleet_primary as fp
    on f.fleet_id = fp.fleet_id
    join fleetseek.phone_number as pn
    on fp.primary_fax_id = pn.phone_number_id
    where 
        m.fax_num != pn.phone_number and
        m.mcs150_date > pn.updated_date_time and
        m.mcs150_date > now() - interval '2 years' and
        f.active_flag = 'Y' and 
        f.total_vehicles between 1 and 19 and
        pn.phone_type_id = 3 and
        m.fax_num is not null

) as sub_q
where phone_number.phone_number_id = sub_q.phone_number_id
""")
conn.commit()

In [74]:
# Phone number changes for regular, toll-free, and fax
# For fleets between 20 and 49 that have the same area code but different number

cur.execute("""
update fleetseek.phone_number
set
    phone_number = sub_q.tel_num,
    updated_date_time = now()
from (
    select 
        m.tel_num, 
        pn.phone_number_id
    from mcmis as m
    join fleetseek.fleet as f
    on m.census_num = f.usdot 
    join fleetseek.fleet_primary as fp
    on f.fleet_id = fp.fleet_id
    join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id
    where 
        left(pn.phone_number, 3) = left(m.tel_num, 3) and
        m.tel_num != pn.phone_number and
        m.mcs150_date > pn.updated_date_time and
        m.mcs150_date > now() - interval '2 years' and
        f.active_flag = 'Y' and 
        f.total_vehicles between 20 and 49 and
        pn.phone_type_id = 1 and 
        m.tel_num not similar to ('800%|888%|877%|866%|855%|844%')
        
) as sub_q
where phone_number.phone_number_id = sub_q.phone_number_id;


-- toll free update
update fleetseek.phone_number
set
    phone_number = sub_q.tel_num,
    updated_date_time = now()
from (
    select 
        m.tel_num, 
        pn.phone_number_id
    from mcmis as m
    join fleetseek.fleet as f
    on m.census_num = f.usdot 
    join fleetseek.fleet_primary as fp
    on f.fleet_id = fp.fleet_id
    join fleetseek.phone_number as pn
    on fp.primary_toll_free_id = pn.phone_number_id
    where 
        left(m.tel_num, 3) = left(pn.phone_number, 3) and
        m.tel_num != pn.phone_number and
        m.mcs150_date > pn.updated_date_time and
        m.mcs150_date > now() - interval '2 years' and
        f.active_flag = 'Y' and 
        f.total_vehicles between 20 and 49 and
        pn.phone_type_id = 2 and
        m.tel_num similar to ('800%|888%|877%|866%|855%|844%') and
        m.tel_num is not null
) as sub_q
where phone_number.phone_number_id = sub_q.phone_number_id;


-- fax number update

update fleetseek.phone_number
set
    phone_number = sub_q.fax_num,
    updated_date_time = now()
from (
    select 
        m.fax_num, 
        pn.phone_number_id,
        pn.phone_type_id

    from mcmis as m
    join fleetseek.fleet as f
    on m.census_num = f.usdot 
    join fleetseek.fleet_primary as fp
    on f.fleet_id = fp.fleet_id
    join fleetseek.phone_number as pn
    on fp.primary_fax_id = pn.phone_number_id
    where 
        left(m.fax_num, 3) = left(pn.phone_number, 3) and
        m.fax_num != pn.phone_number and
        m.mcs150_date > pn.updated_date_time and
        m.mcs150_date > now() - interval '2 years' and
        f.active_flag = 'Y' and 
        f.total_vehicles between 20 and 49 and
        pn.phone_type_id = 3

) as sub_q
where phone_number.phone_number_id = sub_q.phone_number_id

""")

## Email changes

#### 2/25/2020 removing auto updates for emails

In [75]:
# # No update date on this table
# cur.execute("""
# update fleetseek.email
#     set email_address = sub_q.mcmis_email

# from (

#     select lower(m.email_address) as mcmis_email, lower(e.email_address) as current_email, m.mcs150_date, f.update_date, e.email_id
#     from mcmis as m
#     join fleetseek.fleet as f
#     on m.census_num = f.usdot 
#     join fleetseek.fleet_primary as fp
#     on f.fleet_id = fp.fleet_id
#     join fleetseek.email as e
#     on fp.primary_email_id = e.email_id
#     where 
#         lower(btrim(e.email_address)) != lower(btrim(m.email_address)) and
#         m.mcs150_date > f.update_date and
#         m.mcs150_date > now() - interval '2 years' and
#         f.active_flag = 'Y' and 
#         f.total_vehicles between 1 and 19
# ) as sub_q
# where email.email_id = sub_q.email_id and
# email.email_address <> sub_q.mcmis_email

# """)
# conn.commit()

## Contact updates 

#### Contacts will not be deleted but rather different ones will be added

In [ ]:
cur.execute("""
alter table fleetseek.contact
add column census_num integer;
""")
conn.commit()

In [82]:
# Go into contact and fleet_contact table
# This query uses another CTE to insert into contact as well as fleet_contact
# Uses fuzzy matching with 'similarity' and sets the cutoff at anything less than 0.5

cur.execute("""
with inserted as 
(
    insert into fleetseek.contact (first_name, middle_name, last_name, suffix, census_num)

    select 
        sub_q.first_name,
        sub_q.middle_name,
        sub_q.last_name,
        sub_q.suffix,
        sub_q.census_num
    from 
    (

        select 
        initcap(split_part(a.company_rep1a, ' ', 1)) as first_name,
        case
            when (a.count_of_spaces = 3 and (a.company_rep1a ilike '%III%' or a.company_rep1a ilike '%III')) then initcap(split_part(a.company_rep1a, ' ', 2))
            when (a.count_of_spaces = 3 and (a.company_rep1a not ilike '%jr' and a.company_rep1a not ilike '%jr.') and (a.company_rep1a not ilike '%sr' and a.company_rep1a not ilike '%sr.')) then initcap(concat(split_part(a.company_rep1a, ' ', 2), ' ',  split_part(a.company_rep1a, ' ', 3)))
            when (a.count_of_spaces = 3) then initcap(split_part(a.company_rep1a, ' ', 2))
            when (a.count_of_spaces = 2 and a.company_rep1a ilike '%III%') then null
            when (a.count_of_spaces = 2 and (a.company_rep1a not ilike '%jr' and a.company_rep1a not ilike '%jr.') and (a.company_rep1a not ilike '%sr' and a.company_rep1a not ilike '%sr.')) then initcap(split_part(a.company_rep1a, ' ', 2))
        end as middle_name,

        case 
            when (a.count_of_spaces = 3 and (a.company_rep1a ilike '%III%' or a.company_rep1a ilike '%III')) then initcap(split_part(a.company_rep1a, ' ', 3))
            when (a.count_of_spaces = 3 and (a.company_rep1a not ilike '%jr' and a.company_rep1a not ilike '%jr.') and (a.company_rep1a not ilike '%sr' and a.company_rep1a not ilike '%sr.')) then initcap(split_part(a.company_rep1a, ' ', 4))
            when (a.count_of_spaces = 1) then initcap(split_part(a.company_rep1a, ' ', 2)) 
            when (a.count_of_spaces = 2 and company_rep1a ilike '%III%') then initcap(split_part(a.company_rep1a, ' ', 2))
            when (a.count_of_spaces = 2 and (a.company_rep1a ilike '%jr%' or a.company_rep1a ilike '%sr%')) then initcap(split_part(a.company_rep1a, ' ', 2))
            when (a.count_of_spaces = 2 and (a.company_rep1a not ilike '%jr%' or a.company_rep1a not ilike '%sr%')) then initcap(split_part(a.company_rep1a, ' ', 3))
            when (a.count_of_spaces = 3) then initcap(split_part(a.company_rep1a, ' ', 3))
        end as last_name,

        case
            when (a.count_of_spaces = 3 and (a.company_rep1a ilike '%III%' or a.company_rep1a ilike '%III')) then split_part(a.company_rep1a, ' ', 4)
            when (a.count_of_spaces = 3 and (a.company_rep1a not ilike '%jr' and a.company_rep1a not ilike '%jr.') and (a.company_rep1a not ilike '%sr' and a.company_rep1a not ilike '%sr.')) then null
            when (a.count_of_spaces = 3) then split_part(a.company_rep1a, ' ', 4)
            when (a.count_of_spaces = 2 and a.company_rep1a ilike '%jr%') then split_part(a.company_rep1a, ' ', 3)
            when (a.count_of_spaces = 2 and a.company_rep1a ilike '%sr%') then split_part(a.company_rep1a, ' ', 3)
            when (a.count_of_spaces = 2 and a.company_rep1a ilike '%III%') then split_part(a.company_rep1a, ' ', 3)
        end as suffix,
        a.census_num,
        a.contact_id,
        a.company_rep1,
        a.company_rep2


        FROM (
            SELECT

                b.company_rep1_remove_space as company_rep1a,
                b.company_rep2_remove_space as company_rep2a,
                length(b.company_rep1_remove_space) - length(replace(b.company_rep1_remove_space,' ', '')) AS count_of_spaces,
                length(b.company_rep2_remove_space) - length(replace(b.company_rep2_remove_space, ' ', '')) as count_of_spaces2,
                b.census_num,
                b.contact_id,
                b.fleet_id,
                b.company_rep1,
                b.company_rep2


            FROM (
                    select 
                        m.census_num,
                        c.contact_id,
                        f.fleet_id,
                        m.company_rep1,
                        m.company_rep2,
                        replace(m.company_rep1, '  ', ' ') as company_rep1_remove_space,
                        replace(m.company_rep2, '  ', ' ') as company_rep2_remove_space
                    from mcmis as m
                    join fleetseek.fleet as f
                    on m.census_num = f.usdot
                    join fleetseek.fleet_primary as fp
                    on f.fleet_id = fp.fleet_id
                    join fleetseek.contact as c
                    on fp.primary_contact_id = c.contact_id
                 ) as b

             ) as a


    ) as sub_q
    join fleetseek.fleet as f
    on sub_q.census_num = f.usdot
    join fleetseek.fleet_primary as fp
    on f.fleet_id = fp.fleet_id
    join fleetseek.contact as c
    on fp.primary_contact_id = c.contact_id
    where similarity(concat(upper(c. first_name), ' ', upper(c.middle_name), ' ', upper(c.last_name), ' ', upper(c.suffix)), sub_q.company_rep1) < 0.5 and
    f.total_vehicles between 1 and 49 and 
    f.active_flag = 'Y'

    returning 
        contact_id,
        mcmis_id
)

insert into fleetseek.fleet_contact (fleet_id, contact_id)
select
    mcmis_id,
    contact_id
from inserted;

alter table fleetseek.contact
drop column mcmis_id;

""")
conn.commit()
conn.close()